<a href="https://colab.research.google.com/github/Jonathan-code-hub/Many-Mini-OR-Problems/blob/main/LinearProgramming/LP_Simplex_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Simplex Algorithm

In [4]:
import numpy as np

class Simplex:
    def __init__(self, c, A, b):
        """
        Initialize the simplex algorithm.
        Maximize:    c^T x
        Subject to:  A x <= b, x >= 0

        c: objective coefficients (1D array)
        A: constraint coefficients (2D array)
        b: RHS values (1D array)
        """
        self.c = np.array(c, dtype=float)
        self.A = np.array(A, dtype=float)
        self.b = np.array(b, dtype=float)

        self.m, self.n = self.A.shape

        # Build initial tableau #
        self.tableau = np.zeros((self.m + 1, self.n + self.m + 1))
        self.tableau[:self.m, :self.n] = self.A
        self.tableau[:self.m, self.n:self.n + self.m] = np.eye(self.m)
        self.tableau[:self.m, -1] = self.b
        self.tableau[-1, :self.n] = -self.c

    def pivot(self, row, col):
        """Perform pivot on tableau at given row, col."""
        self.tableau[row] /= self.tableau[row, col]
        for r in range(len(self.tableau)):
            if r != row:
                self.tableau[r] -= self.tableau[r, col] * self.tableau[row]

    def solve(self):
        while True:
            # Find entering variable (most negative coefficient in last row) #
            col = np.argmin(self.tableau[-1, :-1])
            if self.tableau[-1, col] >= 0:
                break  # Optimal solution found

            # Find leaving variable (minimum ratio test) #
            ratios = []
            for i in range(self.m):
                if self.tableau[i, col] > 0:
                    ratios.append(self.tableau[i, -1] / self.tableau[i, col])
                else:
                    ratios.append(np.inf)

            row = np.argmin(ratios)
            if ratios[row] == np.inf:
                raise Exception("Problem is unbounded!")

            self.pivot(row, col)

        # Extract solution #
        solution = np.zeros(self.n)
        for j in range(self.n):
            col = self.tableau[:self.m, j]
            if np.count_nonzero(col) == 1 and np.isclose(np.max(col), 1):
                row = np.where(col == 1)[0][0]
                solution[j] = self.tableau[row, -1]

        return solution, self.tableau[-1, -1]


# Example problem:            #
# Maximize z = 3x1 + 2x2      #
# Subject to: x1 + x2 <= 4    #
#             x1 <= 2         #
#             x2 <= 3         #
c = [3, 2]
A = [[1, 1],
     [1, 0],
     [0, 1]]
b = [4, 2, 3]

solver = Simplex(c, A, b)
solution, optimum = solver.solve()
print("Optimal solution:", solution)
print("Optimal value:", optimum)


Optimal solution: [2. 2.]
Optimal value: 10.0
